In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

import numpy as np


from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV





In [2]:
data = pd.read_csv('final_wec_data.csv')

/tmp/ipykernel_1520/3317488320.py:1: DtypeWarning: Columns (26,31,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('final_wec_data.csv')


In [3]:
# data = data.drop('Unnamed: 0', axis=1)
# #  Dropping the extra index col

# data = data[data['year'] >= 2017].reset_index(drop=True)
# # # Dropping data from prior to 2017 as driver_ratings data was not available

# data = data.drop(['lap_number', 'car_number', 'lap_number', 'driver_number', 'lap_time', 'elapsed', 'hour', 's1_large', 's2_large', 's3_large', 'driver_name', 'pit_time', 'group', 'team', 'manufacturer', 'season', 'vehicle', 'team_no','lap_time_ms', 'engine', 'driver_stint',
#            'team_stint', 'team_stint_no', 'interval_ms', 'interval', 'elapsed_ms', 'position', 'gap', 'elapsed_s', ], axis=1)
# # Initial columns dropped because not needed/not useable for modeling. Remaining columns will be further assessed 

In [4]:
def df_for_ml(df):

    df = df.drop('Unnamed: 0', axis=1)
    df = data[data['year'] >= 2017].reset_index(drop=True)
    # Dropping data from prior to 2017 as driver_ratings data was not available
    df = df.drop(['lap_number', 'car_number', 'lap_number', 'driver_number', 'lap_time', 'elapsed', 'hour', 's1_large', 's2_large', 's3_large', 'driver_name', 'pit_time', 'group', 'team', 'manufacturer', 'season', 'vehicle', 'team_no','lap_time_ms', 'engine', 'driver_stint',
           'team_stint', 'team_stint_no', 'interval_ms', 'interval', 'elapsed_ms', 'position', 'gap', 'elapsed_s'], axis=1)
    # Initial columns dropped because not needed/not useable for modeling. Remaining columns will be further assessed 
    
    return df




In [5]:
data = df_for_ml(data)

In [6]:
def fixing_kept_cols(df):
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].fillna(0)
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].replace('B', 1)
#   B represents a car crossing the start/finish line in pitlane, NaN means the car was on track. Values are replaced with binary 0 for on-track, 1 for in-pit

    df = df[df['class']!='INNOVATIVE CAR']
#     removing the innovative class to eliminate outlier data
    
    
    df['circuit'] = df['circuit'].replace({'SPA FRANCORCHAMPS':'Spa', 'BAHRAIN INTERNATIONAL CIRCUIT':'Bahrain', 'FUJI SPEEDWAY':'Fuji', 'SHANGHAI INTERNATIONAL CIRCUIT':'Shanghai', 'CIRCUIT OF THE AMERICAS':'COTA',
    'AUTODROMO HERMANOS RODRIGUEZ':'Mexico', 'LE MANS 2018':'Le Mans', 'SPA FRANCORCHAMPS 2019':'Spa','LE MANS 2019':'Le Mans', 'BAHRAIN INTERNATIONAL CIRCUIT 2019': 'Bahrain',
    'BAHRAIN INTERNATIONAL CIRCUIT 2020':'Bahrain', 'AUTODROMO DO ALGARVE':'Portimao', 'AUTODROMO NAZIONALE DI MONZA':'Monza', 'BAHRAIN INTERNATIONAL CIRCUIT 6 HOURS':'Bahrain', 'BAHRAIN INTERNATIONAL CIRCUIT 8 HOURS':'Bahrain'})
    df['circuit'] = df['circuit'].str.lower() 
#     Replacing the long-form and/or year appended circuit names with the common-use short versions. Also changes all to lower-case


    df = df.drop('flag_at_fl', axis=1)
#     data in this column only exists for season 2022, also some value meanings can not be determined.

    df = df.dropna().reset_index(drop=True)
    # Dropping NaN values to remove rows with missing data
    #12.68% of laps were run by drivers for which we have no rating info
    #otherwise only small percentages of nan values exist in other columns 
    #nans can be safely dropped without significant effect. 
    

    return df


In [7]:
data = fixing_kept_cols(data)

/tmp/ipykernel_1520/2614309269.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['circuit'] = df['circuit'].replace({'SPA FRANCORCHAMPS':'Spa', 'BAHRAIN INTERNATIONAL CIRCUIT':'Bahrain', 'FUJI SPEEDWAY':'Fuji', 'SHANGHAI INTERNATIONAL CIRCUIT':'Shanghai', 'CIRCUIT OF THE AMERICAS':'COTA',
/tmp/ipykernel_1520/2614309269.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['circuit'] = df['circuit'].str.lower()


In [8]:
# data.isna().sum()/len(data)*100

#small percentages of nan values can be safely dropped without significant effect. 
#12.68% of laps were run by a driver for which we have no rating info

In [9]:
# data = data.dropna().reset_index(drop=True)
# # Dropping NaN values to remove rows with missing data

# #This will be included into the fixing_kept_cols function when the code is refactored

In [10]:
# data.isna().sum()/len(data)*100

In [11]:
# data.head()

In [12]:
# data.dtypes

In [13]:
# data.describe()

In [14]:
# data.info()

In [15]:
# data['rating'].value_counts()

In [16]:
# data['circuit'].unique()

In [17]:
# data['class'].unique()

In [18]:
# obs = data.select_dtypes(object).drop('rating', axis=1)

# class_one_hot = pd.get_dummies(obs['class'], prefix='class')

# circuit_one_hot = pd.get_dummies(obs['circuit'], prefix='circuit')

# ob_feats = pd.concat([class_one_hot, circuit_one_hot], axis=1)

# ob_feats

In [19]:
# cont_nums = data.select_dtypes('number')
# cont_nums = cont_nums.drop(['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position'], axis=1)

In [20]:
# disc_nums = data[['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position']]
# disc_nums

In [21]:
def separate_df(df):
    cont_nums = df.select_dtypes('number')
    cont_nums = cont_nums.drop(['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position'], axis=1)
    
    disc_nums = df[['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position']]
    
    obs = data.select_dtypes(object).drop('rating', axis=1)

    target = df['rating']
    
    return cont_nums, disc_nums, obs, target



In [22]:
def encode_obs(obs):
    class_one_hot = pd.get_dummies(obs['class'], prefix='class')
    circuit_one_hot = pd.get_dummies(obs['circuit'], prefix='circuit')
    
    ob_feats = pd.concat([class_one_hot, circuit_one_hot], axis=1)
    
    return ob_feats


In [23]:
cont_nums, disc_nums, obs, target = separate_df(data)

In [24]:
# obs

In [25]:
ob_feats = encode_obs(obs)

In [26]:
# ob_feats.head()

In [27]:
# s_scaler = StandardScaler()
# num_s_scal = s_scaler.fit_transform(nums)
# num_s_scal = pd.DataFrame(num_s_scal)
# num_s_scal

In [28]:
# mm_scaler = MinMaxScaler()
# num_mm_scal = mm_scaler.fit_transform(nums)
# num_mm_scal = pd.DataFrame(num_mm_scal)
# num_mm_scal

In [29]:
# abs_scaler = MaxAbsScaler()
# num_abs_scal = abs_scaler.fit_transform(nums)
# num_abs_scal = pd.DataFrame(num_abs_scal)
# num_abs_scal

In [30]:
def make_scaled_sets(df):
    from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
    
    scales = []
    
    s_scaler = StandardScaler()
    num_s_scal = s_scaler.fit_transform(df)
    num_s_scal = pd.DataFrame(num_s_scal)
    num_s_scal.columns = num_s_scal.columns.astype(str)
    num_s_scal.name = 'StandardScaler'
    scales.append(num_s_scal)
    
    mm_scaler = MinMaxScaler()
    num_mm_scal = mm_scaler.fit_transform(df)
    num_mm_scal = pd.DataFrame(num_mm_scal)
    num_mm_scal.columns = num_mm_scal.columns.astype(str)
    num_mm_scal.name = 'MinMaxScaler'
    scales.append(num_mm_scal)
    
    abs_scaler = MaxAbsScaler()
    num_abs_scal = abs_scaler.fit_transform(df)
    num_abs_scal = pd.DataFrame(num_abs_scal)
    num_abs_scal.columns = num_abs_scal.columns.astype(str)
    num_abs_scal.name = 'MaxAbsScaler'
    scales.append(num_abs_scal)
    
    
    return scales




In [31]:
scales = make_scaled_sets(cont_nums)


In [32]:
# num_s_scal.head()

In [33]:
# num_mm_scal.head()

In [34]:
# num_abs_scal.head()

In [35]:
features = pd.concat([scales[0], disc_nums, ob_feats], axis=1)



In [36]:
# features

In [37]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 55)
# Making a split for testing purposes. 

In [38]:
# rfc = RandomForestClassifier(max_depth=2, random_state=0)
# rfc.fit(X_train,y_train)
# y_pred = rf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

# # Training/fitting an initial run of the RandomForest Classifiier
# #  Accuracy: 0.48788660717485577
# #Initial model run resulted in poor accuracy. Hyperparameter tuning can be used to improve the accuracy result.

In [39]:


# # define the parameter grid
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [5, 10, 15],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2'] }

# rfc = RandomForestClassifier()
# grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # print the best parameters and score
# print("Best parameters: ", grid_search.best_params_)
# print("Best score: ", grid_search.best_score_)



# Best parameters:  {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
# Best score:  0.785742018337664

# ***After running the grid search function through the model hyperparameters we have acheived a best score of ~78% accuracy. The output best parameters wil be used for this model from here on out. 


In [40]:
# def hyper_tuning_rfc():
#     param_grid_rfc = {'n_estimators': [100, 200, 300], 'max_depth': [5, 10, 15], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['sqrt', 'log2'] }
#     rfc = RandomForestClassifier()
#     grid_search = GridSearchCV(rfc, param_grid=param_grid_rfc, cv=5)
#     grid_search.fit(X_train, y_train)
#     rfc_best_params = grid_search.best_params_
    
    
#     #print the best parameters and score
#     print("Best parameters: ", grid_search.best_params_)
#     print("Best score: ", grid_search.best_score_)

#     return rfc_best_params

# # Running multiple loops of the model to find the optimal set of model parameters to maximize model accuracy

In [41]:

# knn = KNeighborsClassifier()
# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

# # An initial run of the KNN Classifier on the same training data as the RandomForest model reports much higher accuracy
# # Accuracy: 0.7998781493029423
# # Hyper-paramter tuning will be run for this model as well in an attempt to maximize our models accuracy.

In [53]:
def hyper_tuning_knn(X_train, y_train):
    param_grid_knn = {'n_neighbors': (1,10, 1), 'leaf_size': (20,40,1), 'p': (1,2), 'weights': ('uniform', 'distance'), 'metric': ('minkowski', 'chebyshev')}
    knn = KNeighborsClassifier()
    grid_search = GridSearchCV(knn, param_grid=param_grid_knn, cv=5)
    grid_search.fit(X_train, y_train)
    knn_best_params = grid_search.best_params_
    
    
    #print the best parameters and score
    print("Best parameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)

    return knn_best_params

# Running multiple loops of the model to find the optimal set of model parameters to maximize model accuracy

In [54]:
knn_best_params = hyper_tuning_knn(X_train, y_train)

Best parameters:  {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Best score:  0.8498700833258669


In [48]:
knn_best_params

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}

In [49]:
knn_best_params = {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}

for i in knn_best_params:
    i.strip('')
    i.replace(:,=)

SyntaxError: invalid syntax (471464264.py, line 5)

In [44]:
# def test_scales_model(scales):
#     for i in range(len(scales)):
#         name = scales[i].name
#         features = pd.concat([scales[i], disc_nums, ob_feats], axis=1)
#         X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 55)
#         rf = RandomForestClassifier(max_depth=15, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=200)
#         rf.fit(X_train,y_train)
#         y_pred = rf.predict(X_test)
#         accuracy = accuracy_score(y_test, y_pred)
#         print(name, "Accuracy:", accuracy)
        
        
        
# ###This function will test the 
        

In [45]:
# test_scales_model(scales)